# 라이브러리 로드 및 기본 세팅

In [1]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, precision_recall_curve

# 데이콘 점수

In [4]:
# Best_score
  # target 1 개수 : 36081
  # self_score : 0.708871
  # date : 2022-01-13 11:08:38
  # score : 0.701834167

In [5]:
# Submit_score
  # target 1 개수 : 37563
  # self_score : 0.7755
  # date : 2022-01-28 11:59:50
  # score : 0.6909665629

# 설정

In [6]:
# cat
# 총 특성 개수 : 62개(타겟 특성 제외)
# 총 제거된 특성 개수 : 7
# 활용 특성 : 55
# 매칭 경우의 수 별로 훈련/예측
# one_hot_max_size = 5
# random_state : 5(range(10, 60, 10))
# iterations = 1000
# early_stopping_rounds = 50
# 순서형 특성 설정(4개)
# probability_threshold 설정

In [7]:
# 순서형 특성 (4개)
# 'person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e'

In [8]:
# 기본 제거 특성(5개)
# 'id',
# 'contents_open_dt',
# 'person_prefer_f',
# 'person_prefer_g',
# 'contents_rn',

In [9]:
# 특성 중요도 낮은 특성(2개)
# 'h_m_match_yn',
# 'h_s_match_yn'

# 데이터 불러오기

In [10]:
# train = pd.read_csv('../data/df_train.csv')
# test = pd.read_csv('../data/df_test.csv')
train = pd.read_csv('/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/data/df_train.csv')
test = pd.read_csv('/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/data/df_test.csv')

# 데이터 확인

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 63 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   id                      501951 non-null  int64 
 1   d_l_match_yn            501951 non-null  bool  
 2   d_m_match_yn            501951 non-null  bool  
 3   d_s_match_yn            501951 non-null  bool  
 4   h_l_match_yn            501951 non-null  bool  
 5   h_m_match_yn            501951 non-null  bool  
 6   h_s_match_yn            501951 non-null  bool  
 7   person_attribute_a      501951 non-null  int64 
 8   person_attribute_a_1    501951 non-null  int64 
 9   person_attribute_b      501951 non-null  int64 
 10  person_prefer_c         501951 non-null  int64 
 11  person_prefer_d_1       501951 non-null  int64 
 12  person_prefer_d_2       501951 non-null  int64 
 13  person_prefer_d_3       501951 non-null  int64 
 14  person_prefer_e         501951 non-n

In [12]:
train.head()

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,0,True,True,True,False,False,False,1,4,3,5,275,370,369,8,1,1,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,2020-01-17 12:09:36,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,316,3,398,94,368,58,422,94,1607,1606,1605,2016
1,1,False,False,False,True,True,False,1,3,4,1,114,181,175,4,1,1,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,2020-06-18 17:48:52,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,216,417,94,400,94,399,94,417,94,1607,1606,1605,2016
2,2,False,False,False,True,False,False,2,0,3,5,464,175,452,3,1,1,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,2020-07-08 20:00:10,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,1,364,48,528,250,366,48,363,48,1599,1595,1572,2016
3,3,False,False,False,True,False,False,2,0,2,5,703,705,704,3,1,1,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,2020-01-13 18:09:34,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,216,379,71,495,226,315,1,381,71,1607,1606,1605,2016
4,4,True,True,True,False,False,False,1,3,4,5,275,370,369,4,1,1,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,2020-03-09 20:39:22,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,483,208,481,208,480,208,381,71,1607,1606,1605,2016


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 62 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      46404 non-null  int64 
 1   d_l_match_yn            46404 non-null  bool  
 2   d_m_match_yn            46404 non-null  bool  
 3   d_s_match_yn            46404 non-null  bool  
 4   h_l_match_yn            46404 non-null  bool  
 5   h_m_match_yn            46404 non-null  bool  
 6   h_s_match_yn            46404 non-null  bool  
 7   person_attribute_a      46404 non-null  int64 
 8   person_attribute_a_1    46404 non-null  int64 
 9   person_attribute_b      46404 non-null  int64 
 10  person_prefer_c         46404 non-null  int64 
 11  person_prefer_d_1       46404 non-null  int64 
 12  person_prefer_d_2       46404 non-null  int64 
 13  person_prefer_d_3       46404 non-null  int64 
 14  person_prefer_e         46404 non-null  int64 
 15  pe

In [14]:
test.head()

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,0,True,False,False,True,True,True,1,1,2,1,857,851,1227,4,1,1,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,393790,236865,2020-12-01 02:24:18,854,853,824,744,847,847,824,744,1227,1227,1227,926,836,831,824,744,528,250,366,48,359,48,528,250,1146,1128,1021,2010
1,1,False,False,False,True,False,False,2,0,2,2,683,1086,662,2,1,1,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,394058,236572,2020-12-17 05:42:53,683,677,659,618,1086,1086,1053,926,662,660,659,618,276,274,274,216,523,250,528,250,359,48,528,250,1610,1606,1605,2016
2,2,True,False,False,True,True,True,2,3,2,1,514,790,1233,0,1,1,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,1002061,704612,2020-12-10 23:33:41,514,509,482,482,784,783,745,744,1227,1227,1227,926,490,490,482,482,453,169,451,169,452,169,453,169,1812,1811,1810,2020
3,3,True,False,False,True,True,True,1,2,2,5,114,181,175,4,1,1,177,170,171,3,3,5,1,1,2,101,150,5,3,177,1000813,704652,2020-12-03 19:44:55,114,109,56,1,175,152,56,1,175,152,56,1,145,123,56,1,453,169,451,169,452,169,453,169,101,100,99,2006
4,4,True,False,False,True,False,False,1,6,4,5,1082,1078,1056,5,1,1,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,111146,704413,2020-12-11 21:24:34,1078,1078,1053,926,1078,1078,1053,926,1056,1054,1053,926,1097,1094,1093,926,454,169,453,169,316,3,453,169,984,980,954,2009


# 필요 데이터 추출

## 훈련에 의미가 적은 열 제거

In [15]:
# 제거된 특성(7개)
# 'id',
# 'contents_open_dt',
# 'person_prefer_f',
# 'person_prefer_g',
# 'contents_rn',
# 'h_m_match_yn',
# 'h_s_match_yn'

In [16]:
# 삭제할 특성
pop_col = ['id', 'contents_open_dt', 'contents_rn', 'person_prefer_f', 'person_prefer_g',
           'h_m_match_yn', 'h_s_match_yn']
len(pop_col)

7

In [17]:
train = train.drop(pop_col, axis=1) 
test = test.drop(pop_col, axis=1)

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 56 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   d_l_match_yn            501951 non-null  bool 
 1   d_m_match_yn            501951 non-null  bool 
 2   d_s_match_yn            501951 non-null  bool 
 3   h_l_match_yn            501951 non-null  bool 
 4   person_attribute_a      501951 non-null  int64
 5   person_attribute_a_1    501951 non-null  int64
 6   person_attribute_b      501951 non-null  int64
 7   person_prefer_c         501951 non-null  int64
 8   person_prefer_d_1       501951 non-null  int64
 9   person_prefer_d_2       501951 non-null  int64
 10  person_prefer_d_3       501951 non-null  int64
 11  person_prefer_e         501951 non-null  int64
 12  person_prefer_h_1       501951 non-null  int64
 13  person_prefer_h_2       501951 non-null  int64
 14  person_prefer_h_3       501951 non-null  int64
 15  

In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 55 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   d_l_match_yn            46404 non-null  bool 
 1   d_m_match_yn            46404 non-null  bool 
 2   d_s_match_yn            46404 non-null  bool 
 3   h_l_match_yn            46404 non-null  bool 
 4   person_attribute_a      46404 non-null  int64
 5   person_attribute_a_1    46404 non-null  int64
 6   person_attribute_b      46404 non-null  int64
 7   person_prefer_c         46404 non-null  int64
 8   person_prefer_d_1       46404 non-null  int64
 9   person_prefer_d_2       46404 non-null  int64
 10  person_prefer_d_3       46404 non-null  int64
 11  person_prefer_e         46404 non-null  int64
 12  person_prefer_h_1       46404 non-null  int64
 13  person_prefer_h_2       46404 non-null  int64
 14  person_prefer_h_3       46404 non-null  int64
 15  contents_attribute_

## 카테고리형 특성 원핫인코딩시 컬럼 명 겹치는 문제 해결위해 특성명 변경

In [20]:
# 카테고리형 특성 원핫인코딩시 컬럼 명 겹치는 문제 해결위해 특성명 변경
train = train.rename(columns={'person_attribute_a_1':'person_attribute_a_a'})
train = train.rename(columns={'contents_attribute_j_1':'contents_attribute_j_a'})
train.head()

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,person_attribute_a,person_attribute_a_a,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_a,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,target,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,True,True,True,False,1,4,3,5,275,370,369,8,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,316,3,398,94,368,58,422,94,1607,1606,1605,2016
1,False,False,False,True,1,3,4,1,114,181,175,4,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,216,417,94,400,94,399,94,417,94,1607,1606,1605,2016
2,False,False,False,True,2,0,3,5,464,175,452,3,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,1,364,48,528,250,366,48,363,48,1599,1595,1572,2016
3,False,False,False,True,2,0,2,5,703,705,704,3,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,216,379,71,495,226,315,1,381,71,1607,1606,1605,2016
4,True,True,True,False,1,3,4,5,275,370,369,4,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,483,208,481,208,480,208,381,71,1607,1606,1605,2016


In [21]:
# 카테고리형 특성 원핫인코딩시 컬럼 명 겹치는 문제 해결위해 특성명 변경
test = test.rename(columns={'person_attribute_a_1':'person_attribute_a_a'})
test = test.rename(columns={'contents_attribute_j_1':'contents_attribute_j_a'})
test.head()

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,person_attribute_a,person_attribute_a_a,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_a,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_1_l,person_prefer_h_2_m,person_prefer_h_2_l,person_prefer_h_3_m,person_prefer_h_3_l,contents_attribute_h_m,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
0,True,False,False,True,1,1,2,1,857,851,1227,4,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,393790,854,853,824,744,847,847,824,744,1227,1227,1227,926,836,831,824,744,528,250,366,48,359,48,528,250,1146,1128,1021,2010
1,False,False,False,True,2,0,2,2,683,1086,662,2,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,394058,683,677,659,618,1086,1086,1053,926,662,660,659,618,276,274,274,216,523,250,528,250,359,48,528,250,1610,1606,1605,2016
2,True,False,False,True,2,3,2,1,514,790,1233,0,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,1002061,514,509,482,482,784,783,745,744,1227,1227,1227,926,490,490,482,482,453,169,451,169,452,169,453,169,1812,1811,1810,2020
3,True,False,False,True,1,2,2,5,114,181,175,4,177,170,171,3,3,5,1,1,2,101,150,5,3,177,1000813,114,109,56,1,175,152,56,1,175,152,56,1,145,123,56,1,453,169,451,169,452,169,453,169,101,100,99,2006
4,True,False,False,True,1,6,4,5,1082,1078,1056,5,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,111146,1078,1078,1053,926,1078,1078,1053,926,1056,1054,1053,926,1097,1094,1093,926,454,169,453,169,316,3,453,169,984,980,954,2009


## 카테고리형 특성으로 지정할 특성

In [22]:
# 원본 데이터에서 순서형 특성
# 'person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e'

In [23]:
cat_f = train.columns.tolist()
numeric_cat = ['person_attribute_a_a', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e']
for col in numeric_cat:
  cat_f.remove(col)
cat_f.remove('target')

In [24]:
# 카데고리형 지정 특성 개수
# 63 - 7 - 5 = 51개
len(cat_f)

51

# 모델 훈련/예측

In [25]:
import itertools

In [26]:
# 단계별로 [0, 1]의 선택지 있는 모든 경우의 수 출력
match_cases = list(itertools.product(*[[0,1] for i in range(2)]))

In [27]:
num_list = [0, 1, 2, 3]

In [29]:
all_train_pred_thr_df = pd.DataFrame()
all_test_pred_thr_df = pd.DataFrame()

for random_state in tqdm(range(10, 60, 10)):

  train_pred_thr_df = pd.DataFrame()
  test_pred_thr_df = pd.DataFrame()
  for num in tqdm(num_list):
    c0, c1 = match_cases[num]
    # 매칭 경우의 수로 구분해서 데이터 셋 가져오기
    split_train = train[(train['d_l_match_yn']==c0)&((train['h_l_match_yn']==c1))]
    split_test = test[(test['d_l_match_yn']==c0)&((test['h_l_match_yn']==c1))]

    split_X = split_train.drop(['target'], axis=1)
    split_y = split_train['target']

    # 데이터 셋 나누기
    X_train, X_val, y_train, y_val = train_test_split(split_X, split_y, random_state=random_state, shuffle=True, stratify=split_y)
    # 모델 생성
    cat = CatBoostClassifier(random_state=42, task_type='GPU', iterations=1000, early_stopping_rounds=50,
                          cat_features=cat_f, eval_metric='F1', one_hot_max_size=2)
    # 모델 훈련
    cat.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)
    
    # precision, recall, threshold 값 계산
    val_proba = cat.predict_proba(X_val)
    precisions, recalls, thresholds = precision_recall_curve(y_val, val_proba[:,1])
    threshold_boundary = thresholds.shape[0]
    # 최고 F1 값을 갖는 threshold 값 구하기
    pre = precisions[:threshold_boundary]
    re = recalls[:threshold_boundary]
    f1 = 2 * pre * re / (pre + re)
    optimal_idx = np.argmax(f1)
    optimal_threshold = thresholds[optimal_idx]

    # 분리된 train 셋의 예측 값
    split_train_proba = cat.predict_proba(split_X)
    split_train_proba_thr = (split_train_proba[:,1] >= optimal_threshold)
    split_train_proba_thr = split_train_proba_thr.astype(int)
    split_train_proba_thr_df = pd.DataFrame(split_train_proba_thr, index=split_X.index)

    # 분리된 test 셋 예측
    split_test_proba = cat.predict_proba(split_test)
    split_test_proba_thr = (split_test_proba[:,1] >= optimal_threshold)
    split_test_proba_thr = split_test_proba_thr.astype(int)
    split_test_proba_thr_df = pd.DataFrame(split_test_proba_thr, index=split_test.index)

    # 분리된 값 합치기
    train_pred_thr_df = pd.concat([train_pred_thr_df, split_train_proba_thr_df])
    test_pred_thr_df = pd.concat([test_pred_thr_df, split_test_proba_thr_df])

    # F1 점수 출력
    pred_raw = cat.predict(X_val)
    pred_proba = cat.predict_proba(X_val)
    pred_thr = (pred_proba[:,1] > optimal_threshold)
    pred_thr = pred_thr.astype(int)
    f1_raw = f1_score(y_val, pred_raw)
    f1_thr = f1_score(y_val, pred_thr)
    print('\n')
    print(f'random_state : {random_state}, split_data_number : {num}')
    print(f'threshold 적용 전 f1 score : {f1_raw}')
    print(f'threshold 적용 후 f1 score : {f1_thr}')
    print('====================================================================================')
    print()

  # random_state별 값 합치기
  all_train_pred_thr_df = pd.concat([all_train_pred_thr_df, train_pred_thr_df], axis=1) 
  all_test_pred_thr_df = pd.concat([all_test_pred_thr_df, test_pred_thr_df], axis=1) 

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.053695
0:	learn: 0.2956123	test: 0.2524851	best: 0.2524851 (0)	total: 195ms	remaining: 3m 15s
100:	learn: 0.4156516	test: 0.4412989	best: 0.4413333 (99)	total: 14.5s	remaining: 2m 9s
200:	learn: 0.4678210	test: 0.4693419	best: 0.4693419 (200)	total: 28.4s	remaining: 1m 52s
300:	learn: 0.4966951	test: 0.4780411	best: 0.4803286 (278)	total: 42.1s	remaining: 1m 37s
400:	learn: 0.5138757	test: 0.4864015	best: 0.4868008 (383)	total: 55.8s	remaining: 1m 23s
bestTest = 0.4873685862
bestIteration = 417
Shrink model to first 418 iterations.



 25%|██▌       | 1/4 [01:10<03:32, 70.88s/it]



random_state : 10, split_data_number : 0
threshold 적용 전 f1 score : 0.4873685862231289
threshold 적용 후 f1 score : 0.6238736646539712

Learning rate set to 0.047669
0:	learn: 0.4460466	test: 0.4701561	best: 0.4701561 (0)	total: 150ms	remaining: 2m 30s
bestTest = 0.4701561065
bestIteration = 0
Shrink model to first 1 iterations.



 50%|█████     | 2/4 [01:30<01:21, 40.73s/it]

...



random_state : 50, split_data_number : 1
threshold 적용 전 f1 score : 0.47013827596351865
threshold 적용 후 f1 score : 0.6040894682798339

Learning rate set to 0.051127
0:	learn: 0.6316634	test: 0.6313514	best: 0.6313514 (0)	total: 173ms	remaining: 2m 52s
100:	learn: 0.6341941	test: 0.6343969	best: 0.6344914 (99)	total: 14.8s	remaining: 2m 11s
bestTest = 0.6385796655
bestIteration = 138
Shrink model to first 139 iterations.



 75%|███████▌  | 3/4 [01:51<00:35, 35.13s/it]



random_state : 50, split_data_number : 2
threshold 적용 전 f1 score : 0.6385796654678312
threshold 적용 후 f1 score : 0.6927877329054113

Learning rate set to 0.045644
0:	learn: 0.7059369	test: 0.7059160	best: 0.7059160 (0)	total: 219ms	remaining: 3m 38s
100:	learn: 0.7165277	test: 0.7235601	best: 0.7236852 (98)	total: 15.2s	remaining: 2m 15s
bestTest = 0.7240540475
bestIteration = 116
Shrink model to first 117 iterations.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide

100%|██████████| 5/5 [18:31<00:00, 222.33s/it]



random_state : 50, split_data_number : 3
threshold 적용 전 f1 score : 0.7240540475444976
threshold 적용 후 f1 score : 0.0



## 특성 명명

In [30]:
col_names = []
for n in range(10,60,10):
  col_names.append(str(n))

In [31]:
all_train_pred_thr_df.columns = col_names
all_test_pred_thr_df.columns = col_names

### 오류난 특성 제거

In [ ]:
# error_col = ['20', '30']
# mod_all_train_pred_thr_df = all_train_pred_thr_df.drop(error_col, axis=1)
# mod_all_test_pred_thr_df = all_test_pred_thr_df.drop(error_col, axis=1)

In [ ]:
# mod_all_train_pred_thr_df['mean'] = mod_all_train_pred_thr_df.mean(axis=1)
# mod_all_test_pred_thr_df['mean'] = mod_all_test_pred_thr_df.mean(axis=1)
# mod_all_train_pred_thr_df = mod_all_train_pred_thr_df.sort_index()
# mod_all_test_pred_thr_df = mod_all_test_pred_thr_df.sort_index()

In [32]:
all_train_pred_thr_df['mean'] = all_train_pred_thr_df.mean(axis=1)
all_test_pred_thr_df['mean'] = all_test_pred_thr_df.mean(axis=1)
all_train_pred_thr_df = all_train_pred_thr_df.sort_index()
all_test_pred_thr_df = all_test_pred_thr_df.sort_index()

## 결과 종합

In [ ]:
# mod_all_train_pred_thr_df

,10,40,50,mean
0,1,1,1,1.000000
1,0,0,0,0.000000
2,1,0,1,0.666667
3,1,0,1,0.666667
4,1,1,0,0.666667
...,...,...,...,...
501946,1,0,1,0.666667
501947,1,1,1,1.000000
501948,1,1,1,1.000000
501949,1,1,1,1.000000


In [ ]:
# mod_all_test_pred_thr_df

,10,40,50,mean
0,1,1,1,1.000000
1,1,0,1,0.666667
2,1,1,1,1.000000
3,0,0,0,0.000000
4,1,1,1,1.000000
...,...,...,...,...
46399,1,1,1,1.000000
46400,1,1,1,1.000000
46401,1,1,1,1.000000
46402,1,1,1,1.000000


In [33]:
all_train_pred_thr_df

,10,20,30,40,50,mean
0,1,1,1,1,1,1.0
1,0,0,0,0,0,0.0
2,1,0,0,0,1,0.4
3,1,0,0,1,1,0.6
4,0,0,0,0,0,0.0
...,...,...,...,...,...,...
501946,1,0,1,1,1,0.8
501947,1,1,0,1,0,0.6
501948,1,1,0,1,0,0.6
501949,1,1,0,1,0,0.6


In [34]:
all_test_pred_thr_df

,10,20,30,40,50,mean
0,1,1,0,1,0,0.6
1,1,0,0,0,1,0.4
2,1,1,0,1,0,0.6
3,0,0,0,0,0,0.0
4,1,1,0,0,0,0.4
...,...,...,...,...,...,...
46399,1,1,1,1,1,1.0
46400,1,1,0,1,0,0.6
46401,1,1,0,1,0,0.6
46402,1,1,0,1,0,0.6


In [35]:
train_pred = (all_train_pred_thr_df['mean'] > 0.5)
train_pred = train_pred.astype(int)

## 자체평가

In [36]:
f1_score(train['target'], train_pred)

0.7755131888560992

## test 예측

In [37]:
test_pred = (all_test_pred_thr_df['mean'] > 0.5)
test_pred = test_pred.astype(int)

In [38]:
np.unique(test_pred, return_counts=True)

(array([0, 1]), array([ 8841, 37563]))

# 제출파일 불러오기

In [39]:
# sub = pd.read_csv('../data/sample_submission.csv')
sub = pd.read_csv('/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/data/sample_submission.csv')
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      46404 non-null  int64
 1   target  46404 non-null  int64
dtypes: int64(2)
memory usage: 725.2 KB


# 제출파일 생성

In [40]:
sub['target'] = test_pred
sub['target'].value_counts()

1    37563
0     8841
Name: target, dtype: int64

In [41]:
file_ver = '6.8'
# sub.to_csv(f'sub_ver{file_ver}.csv', index=False)
sub.to_csv(f'/content/drive/MyDrive/GitHub/Dacon/TP2(잡케어)/03_모델선정및개선/sub_ver{file_ver}.csv', index=False)

# 코드 마지막